In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cấu hình đường dẫn đầu vào/đầu ra
input_path = 'folder_clean_visual/telco_clean.csv'
output_folder = 'folder_standardized'
os.makedirs(output_folder, exist_ok=True)
df = None

print("--- KHỞI TẠO PROCESSING ---")

--- KHỞI TẠO PROCESSING ---


In [3]:
def load_clean_data():
    """
    Đọc dữ liệu đã làm sạch từ bước EDA.
    
    Raises:
        FileNotFoundError: Nếu không tìm thấy file dữ liệu sạch
    """
    global df
    print("\n1. LOAD CLEAN DATA")
    if os.path.exists(input_path):
        df = pd.read_csv(input_path)
        print(f"   - Đã đọc dữ liệu: {df.shape}")
    else:
        raise FileNotFoundError(f"   ✗ Không tìm thấy file {input_path}. Hãy chạy EDA.ipynb trước!")

load_clean_data()


1. LOAD CLEAN DATA
   - Đã đọc dữ liệu: (7021, 20)


# FEATURE ENGINEERING

In [4]:
def create_new_features():
    """
    Tạo các đặc trưng mới (Feature Engineering) để cải thiện khả năng dự báo.
    Bao gồm: Has_Family, Num_Services, Payment_Type, Avg_Charges_Per_Service
    """
    global df
    print("\n2. FEATURE ENGINEERING")
    
    # Has_Family: Khách hàng có gia đình (Partner hoặc Dependents)
    df['Has_Family'] = ((df['Partner'] == 'Yes') | (df['Dependents'] == 'Yes')).astype(int)
    
    # Num_Services: Tổng số dịch vụ gia tăng mà khách hàng sử dụng
    services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                'TechSupport', 'StreamingTV', 'StreamingMovies']
    df['Num_Services'] = (df[services] == 'Yes').sum(axis=1)
    
    # Payment_Type: Phân loại hình thức thanh toán (Automatic vs Manual)
    df['Payment_Type'] = df['PaymentMethod'].apply(
        lambda x: 'Automatic' if 'automatic' in x.lower() else 'Manual'
    )
    
    # Avg_Charges_Per_Service: Cước phí trung bình trên mỗi dịch vụ (+1 tránh chia cho 0)
    df['Avg_Charges_Per_Service'] = df['MonthlyCharges'] / (df['Num_Services'] + 1)

    print("   - Thêm các cột: Has_Family, Num_Services, Payment_Type, Avg_Charges_Per_Service")
    print(f"   - Kích thước dữ liệu: {df.shape}")

create_new_features()


2. FEATURE ENGINEERING
   - Thêm các cột: Has_Family, Num_Services, Payment_Type, Avg_Charges_Per_Service
   - Kích thước dữ liệu: (7021, 24)


# ENCODING DATA

In [5]:
def encode_data():
    """
    Thực hiện Label Encoding cho biến mục tiêu (Churn) 
    và One-Hot Encoding cho các biến phân loại còn lại.
    """
    global df
    print("\n3. ENCODING DATA")
    
    # Label Encoding cho biến mục tiêu Churn (Yes=1, No=0)
    le = LabelEncoder()
    df['Churn'] = le.fit_transform(df['Churn'])
    joblib.dump(le, os.path.join(output_folder, 'label_encoder_churn.pkl'))
    print(f"   - Encode 'Churn': Yes=1, No=0")

    # One-Hot Encoding cho các biến phân loại còn lại
    categorical_cols = df.select_dtypes(include=['object']).columns
    print(f"   ℹ Tìm thấy {len(categorical_cols)} cột cần One-Hot Encoding")
    
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print(f"   - Shape sau One-Hot Encoding: {df.shape}")

encode_data()


3. ENCODING DATA
   - Encode 'Churn': Yes=1, No=0
   ℹ Tìm thấy 16 cột cần One-Hot Encoding
   - Shape sau One-Hot Encoding: (7021, 35)


# TRAIN/TEST SPLIT

In [6]:
X_train = None
X_test = None
y_train = None
y_test = None

def split_data():
    """
    Chia dữ liệu thành tập Train (80%) và Test (20%) trước khi chuẩn hóa.
    Điều này đảm bảo không xảy ra data leakage từ tập Test vào quá trình Training.
    """
    global df, X_train, X_test, y_train, y_test
    print("\n4. TRAIN/TEST SPLIT")
    
    # Tách biến mục tiêu (y) và các đặc trưng (X)
    X = df.drop('Churn', axis=1)
    y = df['Churn']
    
    # Chia tập với random_state=42 để đảm bảo tái tạo được kết quả
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"   - Kích thước tập Train: {X_train.shape}")
    print(f"   - Kích thước tập Test: {X_test.shape}")
    print(f"    Tỷ lệ: {X_train.shape[0] / (X_train.shape[0] + X_test.shape[0]) * 100:.1f}% Train, 20% Test")

split_data()


4. TRAIN/TEST SPLIT
   - Kích thước tập Train: (5616, 34)
   - Kích thước tập Test: (1405, 34)
    Tỷ lệ: 80.0% Train, 20% Test


# SCALE DATA

In [7]:
def scale_data_correctly():
    """
    Chuẩn hóa dữ liệu số bằng MinMax Scaling (giới hạn [0, 1]).
    
    ⚠ Quy tắc quan trọng: Fit trên X_train, Transform trên X_train và X_test
    để tránh data leakage (thông tin từ Test set lọt vào Training).
    """
    global X_train, X_test
    print("\n5. SCALE DATA")
    
    # Danh sách các cột số cần chuẩn hóa
    cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges', 
                     'Num_Services', 'Avg_Charges_Per_Service']
    
    # Chuyển đổi sang float trước khi scale
    X_train[cols_to_scale] = X_train[cols_to_scale].astype(float)
    X_test[cols_to_scale] = X_test[cols_to_scale].astype(float)
    print(f"   - Chuyển {len(cols_to_scale)} cột sang kiểu float")

    # Khởi tạo MinMax Scaler
    scaler = MinMaxScaler()
    
    # Fit và Transform trên tập Train
    X_train.loc[:, cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])
    
    # Chỉ Transform (không Fit) trên tập Test
    X_test.loc[:, cols_to_scale] = scaler.transform(X_test[cols_to_scale])
    
    # Lưu scaler để dùng cho dữ liệu mới sau này
    scaler_path = os.path.join(output_folder, 'minmax_scaler.pkl')
    joblib.dump(scaler, scaler_path)
    
    print(f"   - Chuẩn hóa MinMax thành công (phạm vi [0, 1])")
    print(f"   - Lưu Scaler tại: {scaler_path}")

scale_data_correctly()


5. SCALE DATA
   - Chuyển 5 cột sang kiểu float
   - Chuẩn hóa MinMax thành công (phạm vi [0, 1])
   - Lưu Scaler tại: folder_standardized\minmax_scaler.pkl


# SAVE PROCESSED DATA

In [8]:
def save_processed_files():
    """
    Lưu các tập dữ liệu đã xử lý và các công cụ cần thiết cho bước Training/Prediction.
    
    Files lưu:
    - X_train.csv, X_test.csv: Dữ liệu đặc trưng
    - y_train.csv, y_test.csv: Biến mục tiêu
    - train_columns.pkl: Danh sách cột để đảm bảo khớp khi dự báo trên dữ liệu mới
    """
    global X_train, X_test, y_train, y_test
    print("\n6. SAVE PROCESSED DATA")
    
    # Lưu các tập dữ liệu
    X_train.to_csv(os.path.join(output_folder, 'X_train.csv'), index=False)
    X_test.to_csv(os.path.join(output_folder, 'X_test.csv'), index=False)
    y_train.to_csv(os.path.join(output_folder, 'y_train.csv'), index=False)
    y_test.to_csv(os.path.join(output_folder, 'y_test.csv'), index=False)
    print(f"   - Lưu X_train, X_test, y_train, y_test")
    
    # Lưu danh sách tên cột để dùng cho dữ liệu mới (đảm bảo khớp cột)
    joblib.dump(X_train.columns.tolist(), 
                os.path.join(output_folder, 'train_columns.pkl'))
    print(f"   - Lưu danh sách cột (train_columns.pkl)")
    
    print(f"\n   → Tất cả files đã lưu tại thư mục: {output_folder}")

save_processed_files()


6. SAVE PROCESSED DATA
   - Lưu X_train, X_test, y_train, y_test
   - Lưu danh sách cột (train_columns.pkl)

   → Tất cả files đã lưu tại thư mục: folder_standardized
